In [1]:
# Made by Ida Parkkali
# Heavily based on a Jupiter Notebook from course ELEC-E7852 Computational Design Interaction made by Joongi Shin.

import ast
import openai
import json
import re

# Input your OpenAI API key:
openai.api_key = ''

# Define the model you want to use: https://platform.openai.com/docs/models
# It is a good practice to test with a specific version of LLM.
llm_model = "gpt-4o-2024-08-06"  # or gpt-3.5-turbo

# We are using OpenAI chat.completion: https://platform.openai.com/docs/guides/text-generation
def generate_response(_prompts):
    
    chat_completion = openai.chat.completions.create(
        model=llm_model,
        messages=_prompts,
    )
    print("\n---------------------------\n")
    print("\n##### Input prompt #####\n")
    for i in _prompts:
        print(i["content"])
    output = chat_completion.choices[0].message.content
    print(f"\n##### {llm_model} output #####\n")
    print(f"{output}")
    
    return output

# Let's look inside the dataset 

with open("email_dataset.txt", "r", encoding='utf-8') as file:
    email_dataset = json.load(file)
    
# For example...
user_model = {
    "Sender": "unknown",
    "Sender's job": "unknown",
    "Sender's response time": "unknown",
}

In [2]:
# For example...
task_model = [
    {
        "Recipient": "unknown",
        "Recipient's relationship with the sender": "unknown",
        "Goal of the email": "unknown",
        "Tone of the email": "unknown",
        "Grammatical errors or typos": "unknown",
        "Emojis in the email": "unknown",
        # ...
    }
]

In [3]:
def infer_task_model(_email, _task_model):
    prompts = [
        {
            "role": "system",  # This is to set the role and instrutions for LLM
            "content": "You are an agent designed to auto complete an email.\n"                       
                       "You will be given an \"email\" and previous information about the \"task\".\n" 
        },
        {
            "role": "user",  # This is the input prompt
            "content": "### Email ###\n"
                       f"{_email}\n"
                       "### Task information ###\n"
                       f"{_task_model}\n"
                       "Update the listed information based on the email.\n"
                       "Show only the result in the following format:\n"
                       "{\n" 
                       "    \"Recipient\": \"\",\n"  
                       "    \"Recipient's relationship with the sender\": \"\",\n" 
                       "    \"Sender's goal with the recipient\": \"\",\n" 
                        "    \"Sender's tone for the recipient\": \"\",\n" 
                        "    \"Sender's amount of mistakes in writing\": \"\",\n" 
                        "    \"Sender's amount of emojis in writing\": \"\"\n" 
                       "}\n"
        }
    ]
    
    output = generate_response(prompts)
    
    # Conver to JSON
    start = output.find('{')
    end = output.find('}') + 1
    json_output = json.loads(output[start:end])

    return json_output

In [4]:
def infer_user_model(_email, _user_model):
    prompts = [
        {
            "role": "system",  # This is to set the role and instrutions for LLM
            "content": "You are an agent designed to auto complete an email.\n"                       
                       "You will be given an \"email\" and your previous understanding about the \"sender\".\n" 
        },
        {
            "role": "user",  # This is the input prompt
            "content": "### Email ###\n"
                       f"{_email}\n"
                       "### Sender ###\n"
                       f"{_user_model}\n"
                       "Update the listed information based on the email.\n"
                       "Show only the result in the following format:\n"
                       "{\n"
                       "    \"Sender\": \"\",\n"
                       "    \"Sender's job\": \"\",\n"
                       "    \"Sender's response time\": \"\"\n"
                       "}"
        }
    ]
    
    output = generate_response(prompts)
    
    # Conver to JSON
    start = output.find('{')
    end = output.find('}') + 1
    json_output = json.loads(output[start:end])
    
    return json_output

In [5]:
def update_task_model(_email, _task_model):
    # 1. Check whether the recipient's name exist in the task_model.
    prompts = [
        {
            "role": "system",
            "content": "### Email ###\n"
                       f"{_email}\n"
                       "Who is the email recipient?\n"
                       "Show only the result."
        }
    ]
    recipient_name = generate_response(prompts).strip().lower()
      
    recipient_found = False
    
    # 2. If the recipient does not exist, then add new task_model about the recipient.
    for item in _task_model:
        if item.get("Recipient", "").lower() in recipient_name:
            recipient_found = True
            new_task_model = infer_task_model(_email, item)
            item.update(new_task_model)
            break
    
    # 3. If the recipient exists, then update the task_model of the recipient.
    if not recipient_found:
        new_task_model = infer_task_model(_email, item)
        _task_model.append(new_task_model)
    
    # See how the task model changes
    print(f"\n##### Task model #####\n")
    print(_task_model)    

In [6]:
def update_user_model(_email, _user_model):
    # 1. Infer sender's name from the email.
    prompts = [
        {
            "role": "system",
            "content": "### Email ###\n"
                       f"{_email}\n"
                       "Who is the sender of this email?\n"
                       "Show only the result."
        }
    ]
    sender_name = generate_response(prompts).strip().lower()
    
    sender_found = False
    
    # 2. Check whether the sender's name exists in the user_model.
    if _user_model.get("Sender", "").lower() == sender_name:
        sender_found = True
        updated_user_model = infer_user_model(_email, _user_model)
        _user_model.update(updated_user_model)
    
    # 3. If the sender does not exist, then create a new user model.
    if not sender_found:
        new_user_model = infer_user_model(_email, _user_model)
        _user_model.update(new_user_model)
    
    # See how the user model changes
    print(f"\n##### Updated User Model #####\n")
    print(_user_model)


In [7]:
for email in email_dataset:
    update_user_model(email, user_model)  # Update user model
    update_task_model(email, task_model)  # Update task model


---------------------------


##### Input prompt #####

### Email ###
Dear Bob,

I hope this email finds you well. I am writing to update you on the progress of our user interaction study. We have made significant advances, which I beleve will enhance our understanding of user behavior.

Additionally, I am planning a scuba diving trip once we wrap up the current phase. I would be glad to share insights from the dive if you're interested.

Kind regars,

Tom
Who is the sender of this email?
Show only the result.

##### gpt-4o-2024-08-06 output #####

Tom

---------------------------


##### Input prompt #####

You are an agent designed to auto complete an email.
You will be given an "email" and your previous understanding about the "sender".

### Email ###
Dear Bob,

I hope this email finds you well. I am writing to update you on the progress of our user interaction study. We have made significant advances, which I beleve will enhance our understanding of user behavior.

Additionally, I

In [17]:
recipient = input("Email recipient: ") # e.g., Bob
subject = input("Email subject: ")  # e.g., Request a promotion

prompts = [
    {
        "role": "system",  # This is to set the role and instrutions for LLM
        "content": "You are an agent designed to auto complete an email.\n"
                   "You will be given a \"sender\", \"task information\", \"recipient\" and \"subject\" of an email.\n"
    },
    {
        "role": "user",  # This is the input prompt
        "content": f"Sender: {user_model}\n"
                   f"Write an email to \'{recipient}\' about \'{subject}\' based on the task information below:\n"
                   f"Task information: {task_model}\n"
                   "Show only the result."
    }
]
output = generate_response(prompts)

Email recipient:  Mom
Email subject:  Lunch



---------------------------


##### Input prompt #####

You are an agent designed to auto complete an email.
You will be given a "sender", "task information", "recipient" and "subject" of an email.

Sender: {'Sender': 'Tom', "Sender's job": 'researcher in Human-Computer Interaction, enjoys scuba diving and is actively engaged in HCI research. He finds parallels between his research and scuba diving in terms of attention to detail.', "Sender's response time": 'delayed response due to being busy with ongoing research and diving activities, as well as the potential presentation of work.'}
Write an email to 'Mom' about 'Lunch' based on the task information below:
Task information: [{'Recipient': 'unknown', "Recipient's relationship with the sender": 'unknown', 'Goal of the email': 'unknown', 'Tone of the email': 'unknown', 'Grammatical errors or typos': 'unknown', 'Emojis in the email': 'unknown'}, {'Recipient': 'Bob', "Recipient's relationship with the sender": 'colleague or collaborator

In [18]:
print("User model:")
print("{}".format(user_model))
print("Task model:")
print("{}".format(task_model))

User model:
{'Sender': 'Tom', "Sender's job": 'researcher in Human-Computer Interaction, enjoys scuba diving and is actively engaged in HCI research. He finds parallels between his research and scuba diving in terms of attention to detail.', "Sender's response time": 'delayed response due to being busy with ongoing research and diving activities, as well as the potential presentation of work.'}
Task model:
[{'Recipient': 'unknown', "Recipient's relationship with the sender": 'unknown', 'Goal of the email': 'unknown', 'Tone of the email': 'unknown', 'Grammatical errors or typos': 'unknown', 'Emojis in the email': 'unknown'}, {'Recipient': 'Bob', "Recipient's relationship with the sender": 'colleague or collaborator', "Sender's goal with the recipient": 'update Bob on the latest progress of the HCI project and inquire about specific requests for the project report', "Sender's tone for the recipient": 'professional and friendly', "Sender's amount of mistakes in writing": '9 grammatical er